# Preloto: Ensemble Backtest (Standalone)

Este notebook contém **todo o código fonte** necessário para executar o Backtest de Ensemble do Preloto. 
Não é necessário montar o Google Drive nem fazer upload de arquivos.
Os dados da Mega Sena, Lotofácil e Quina serão baixados automaticamente do repositório público.

## 1. Instalação de Dependências

In [ ]:
!pip install xgboost tensorflow scikit-learn pandas

## 2. Imports e Configurações Globais

In [ ]:
import pandas as pd
import numpy as np
import sys
import random
import statistics
import xgboost as xgb
import tensorflow as tf
import gc
import requests
import io
from abc import ABC, abstractmethod
from typing import List, Dict, Any
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from collections import Counter

# Configurar Pandas para exibir tudo se precisar
pd.set_option('display.max_columns', None)

## 3. Classes Base e Utilitários

In [ ]:
# --- src/loterias/features.py ---
def calculate_sum(numbers: List[int]) -> int:
    return sum(numbers)

def count_odds(numbers: List[int]) -> int:
    return sum(1 for n in numbers if n % 2 != 0)

def count_evens(numbers: List[int]) -> int:
    return sum(1 for n in numbers if n % 2 == 0)

def calculate_spread(numbers: List[int]) -> int:
    if not numbers:
        return 0
    return max(numbers) - min(numbers)

# --- src/loterias/data_manager.py ---
class DataManager:
    @staticmethod
    def load_csv(url: str) -> pd.DataFrame:
        try:
            print(f"Baixando dados de: {url}")
            # Usar requests para garantir download limpo caso pandas falhe direto
            s = requests.get(url).content
            return pd.read_csv(io.StringIO(s.decode('utf-8')))
        except Exception as e:
            print(f"Error loading data from {url}: {e}")
            raise e

# --- src/loterias/base.py ---
class Lottery(ABC):
    def __init__(self, name: str, data_url: str, slug: str):
        self.name = name
        self.data_url = data_url
        self.slug = slug
        self.data = None

    @abstractmethod
    def load_data(self) -> pd.DataFrame:
        pass

    @abstractmethod
    def preprocess_data(self) -> pd.DataFrame:
        pass

class Model(ABC):
    def __init__(self, name: str):
        self.name = name

    @abstractmethod
    def train(self, data: pd.DataFrame):
        pass

    @abstractmethod
    def predict(self, **kwargs) -> list:
        pass

## 4. Modelos de Predição (AI/ML)

In [ ]:
# --- src/loterias/models/monte_carlo.py ---
class MonteCarloModel(Model):
    def __init__(self, range_min: int, range_max: int, draw_count: int):
        super().__init__("Monte Carlo Simulation")
        self.range_min = range_min
        self.range_max = range_max
        self.draw_count = draw_count
        self.sum_stats = {}
        self.odd_probs = {}
        self.spread_stats = {}
        self.trained = False

    def train(self, data: pd.DataFrame):
        sums = []
        odds = []
        spreads = []
        if 'dezenas' in data.columns:
             for _, row in data.iterrows():
                try:
                    draw = row['dezenas']
                    if not isinstance(draw, list):
                        continue
                    draw = [int(x) for x in draw]
                    sums.append(calculate_sum(draw))
                    odds.append(count_odds(draw))
                    spreads.append(calculate_spread(draw))
                except:
                    continue
        
        if not sums:
            print("Warning: No data for Monte Carlo training.")
            return

        self.sum_stats = {
            "mean": statistics.mean(sums),
            "stdev": statistics.stdev(sums) if len(sums) > 1 else 0
        }
        total = len(odds)
        self.odd_probs = {}
        for o in set(odds):
            self.odd_probs[o] = odds.count(o) / total
            
        self.spread_stats = {
            "mean": statistics.mean(spreads),
            "stdev": statistics.stdev(spreads) if len(spreads) > 1 else 0
        }
        self.trained = True

    def predict(self, count: int = None, **kwargs) -> list:
        if not self.trained: return []
        final_count = count if count is not None else self.draw_count
        simulations = 10000 
        valid_draws = []
        min_sum = self.sum_stats['mean'] - 1.5 * self.sum_stats['stdev']
        max_sum = self.sum_stats['mean'] + 1.5 * self.sum_stats['stdev']
        min_spread = self.spread_stats['mean'] - 1.5 * self.spread_stats['stdev']
        max_spread = self.spread_stats['mean'] + 1.5 * self.spread_stats['stdev']
        min_prob = 0.05
        rng = list(range(self.range_min, self.range_max + 1))
        
        for _ in range(simulations):
            draw = random.sample(rng, self.draw_count)
            s = calculate_sum(draw)
            if not (min_sum <= s <= max_sum): continue
            sp = calculate_spread(draw)
            if not (min_spread <= sp <= max_spread): continue
            o = count_odds(draw)
            if self.odd_probs.get(o, 0) < min_prob: continue
            valid_draws.append(draw)
            
        if not valid_draws: return sorted(random.sample(rng, final_count))
        freqs = {n: 0 for n in rng}
        for draw in valid_draws:
            for n in draw: freqs[n] += 1
        sorted_nums = sorted(freqs.items(), key=lambda x: x[1], reverse=True)
        top_numbers = [n for n, c in sorted_nums[:final_count]]
        return sorted(top_numbers)

# --- src/loterias/models/rf_model.py ---
class RandomForestModel(Model):
    def __init__(self, range_min: int, range_max: int, draw_count: int):
        super().__init__("Random Forest Model")
        self.range_min = range_min
        self.range_max = range_max
        self.draw_count = draw_count
        self.model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        self.scaler = StandardScaler()
        self.trained = False

    def train(self, data: pd.DataFrame, **kwargs):
        n_jobs = int(kwargs.get('n_jobs', -1))
        if 'n_estimators' in kwargs:
            try:
                n = int(kwargs['n_estimators'])
                self.model = RandomForestClassifier(n_estimators=n, random_state=42, n_jobs=n_jobs)
                # print(f"Re-initialized Random Forest with {n} estimators...", file=sys.stderr)
            except ValueError: pass
        else:
            self.model.n_jobs = n_jobs

        X = []
        y = []
        total_draws = len(data)
        current_gaps = {n: 0 for n in range(self.range_min, self.range_max + 1)}
        freq_total = {n: 0 for n in range(self.range_min, self.range_max + 1)}
        freq_10 = {n: [] for n in range(self.range_min, self.range_max + 1)}
        start_training_idx = 50
        last_draw_features = [0, 0, 0, 0]
        
        for i, row in data.iterrows():
            drawn_numbers = []
            drawn_set = set(row['dezenas'])
            for col in data.columns:
                 if 'bola' in col or 'dezenas' in col:
                     try: drawn_numbers.append(int(row[col]))
                     except: pass
            
            if i >= start_training_idx:
                ctx_sum, ctx_odd, ctx_even, ctx_spread = last_draw_features
                for n in range(self.range_min, self.range_max + 1):
                    X.append([current_gaps[n], freq_total[n], sum(freq_10[n][-10:]), ctx_sum, ctx_odd, ctx_even, ctx_spread])
                    y.append(1 if n in drawn_set else 0)
            
            for n in range(self.range_min, self.range_max + 1):
                if n in drawn_set:
                    current_gaps[n] = 0
                    freq_total[n] += 1
                    freq_10[n].append(1)
                else:
                    current_gaps[n] += 1
                    freq_10[n].append(0)
            last_draw_features = [
                calculate_sum(drawn_numbers), count_odds(drawn_numbers), 
                count_evens(drawn_numbers), calculate_spread(drawn_numbers)
            ]
        
        if not X: return
        X_scaled = self.scaler.fit_transform(np.array(X))
        self.model.fit(X_scaled, np.array(y))
        self.trained = True
        self.final_gaps = current_gaps
        self.final_freq = freq_total
        self.final_freq10 = freq_10
        self.last_draw_features = last_draw_features

    def predict(self, count: int = None, **kwargs) -> list:
        if not self.trained: raise ValueError("Model not trained")
        final_count = count if count is not None else self.draw_count
        X_next = []
        numbers = []
        ctx_sum, ctx_odd, ctx_even, ctx_spread = self.last_draw_features
        for n in range(self.range_min, self.range_max + 1):
            X_next.append([self.final_gaps[n], self.final_freq[n], sum(self.final_freq10[n][-10:]), ctx_sum, ctx_odd, ctx_even, ctx_spread])
            numbers.append(n)
        
        probs = self.model.predict_proba(self.scaler.transform(np.array(X_next)))[:, 1]
        df = pd.DataFrame({'dezenas': numbers, 'prob': probs})
        return sorted(df.sort_values(by=['prob', 'dezenas'], ascending=[False, True]).head(final_count)['dezenas'].tolist())

# --- src/loterias/models/xgboost_model.py ---
class XGBoostModel(Model):
    def __init__(self, range_min: int, range_max: int, draw_count: int):
        super().__init__("XGBoost")
        self.range_min = range_min
        self.range_max = range_max
        self.draw_count = draw_count
        self.model = xgb.XGBClassifier(
            n_estimators=100, learning_rate=0.1, max_depth=5, 
            random_state=42, n_jobs=-1, eval_metric='logloss'
        )
        self.scaler = StandardScaler()
        self.trained = False

    def train(self, data: pd.DataFrame, **kwargs):
        if 'n_estimators' in kwargs: 
            try: self.model.set_params(n_estimators=int(kwargs['n_estimators']))
            except: pass
        if 'learning_rate' in kwargs:
             try: self.model.set_params(learning_rate=float(kwargs['learning_rate']))
             except: pass
        if 'n_jobs' in kwargs:
             try: self.model.set_params(n_jobs=int(kwargs['n_jobs']))
             except: pass

        # Logic is identical to RF, simplified here for space
        # Replicating feature extraction logic:
        X, y = [], []
        current_gaps = {n: 0 for n in range(self.range_min, self.range_max + 1)}
        freq_total = {n: 0 for n in range(self.range_min, self.range_max + 1)}
        freq_10 = {n: [] for n in range(self.range_min, self.range_max + 1)}
        start_training_idx = 50
        last_draw_features = [0, 0, 0, 0]
        
        for i, row in data.iterrows():
            drawn_numbers = []
            drawn_set = set(row['dezenas'])
            for col in data.columns:
                 if 'bola' in col or 'dezenas' in col:
                     try: drawn_numbers.append(int(row[col]))
                     except: pass
            if i >= start_training_idx:
                ctx_sum, ctx_odd, ctx_even, ctx_spread = last_draw_features
                for n in range(self.range_min, self.range_max + 1):
                    X.append([current_gaps[n], freq_total[n], sum(freq_10[n][-10:]), ctx_sum, ctx_odd, ctx_even, ctx_spread])
                    y.append(1 if n in drawn_set else 0)
            for n in range(self.range_min, self.range_max + 1):
                if n in drawn_set:
                    current_gaps[n] = 0; freq_total[n] += 1; freq_10[n].append(1)
                else:
                    current_gaps[n] += 1; freq_10[n].append(0)
            last_draw_features = [calculate_sum(drawn_numbers), count_odds(drawn_numbers), count_evens(drawn_numbers), calculate_spread(drawn_numbers)]
        
        if not X: return
        X_scaled = self.scaler.fit_transform(np.array(X))
        self.model.fit(X_scaled, np.array(y))
        self.trained = True
        self.final_gaps = current_gaps
        self.final_freq = freq_total
        self.final_freq10 = freq_10
        self.last_draw_features = last_draw_features

    def predict(self, count: int = None, **kwargs) -> list:
        if not self.trained: return []
        final_count = count if count is not None else self.draw_count
        X_next, numbers = [], []
        ctx_sum, ctx_odd, ctx_even, ctx_spread = self.last_draw_features
        for n in range(self.range_min, self.range_max + 1):
            X_next.append([self.final_gaps[n], self.final_freq[n], sum(self.final_freq10[n][-10:]), ctx_sum, ctx_odd, ctx_even, ctx_spread])
            numbers.append(n)
        probs = self.model.predict_proba(self.scaler.transform(np.array(X_next)))[:, 1]
        df = pd.DataFrame({'dezenas': numbers, 'prob': probs})
        return sorted(df.sort_values(by=['prob', 'dezenas'], ascending=[False, True]).head(final_count)['dezenas'].tolist())

# --- src/loterias/models/lstm_model.py ---
class LSTMModel(Model):
    def __init__(self, range_min: int, range_max: int, draw_count: int):
        super().__init__("LSTM Deep Learning")
        self.range_min = range_min
        self.range_max = range_max
        self.draw_count = draw_count
        self.model = None
        self.window_size = 10 
        self.units = 128
        self.input_size = (self.range_max + 1) + 4

    def _prepare_sequences(self, data: pd.DataFrame):
        ball_cols = [c for c in data.columns if 'bola' in c.lower() or 'dezenas' in c.lower()]
        draws = data[ball_cols].values.tolist()
        X, y = [], []
        rows = len(draws)
        if rows <= self.window_size: return np.array([]), np.array([])
        for i in range(rows - self.window_size):
            window = draws[i : i + self.window_size]
            target = draws[i + self.window_size]
            X.append(self._draws_to_multihot(window))
            y.append(self._draw_to_multihot(target, include_features=False))
        return np.array(X), np.array(y)

    def _draw_to_multihot(self, draw, include_features=True):
        vec = np.zeros(self.range_max + 1)
        valid_numbers = []
        for num in draw:
            try:
                n = int(num)
                if 0 <= n <= self.range_max:
                    vec[n] = 1.0
                    valid_numbers.append(n)
            except: pass
        if not include_features: return vec
        max_sum = self.range_max * self.draw_count
        s = calculate_sum(valid_numbers) / max_sum
        o = count_odds(valid_numbers) / self.draw_count
        e = count_evens(valid_numbers) / self.draw_count
        sp = calculate_spread(valid_numbers) / self.range_max
        return np.concatenate([vec, np.array([s, o, e, sp])])

    def _draws_to_multihot(self, draws):
        return np.array([self._draw_to_multihot(d, include_features=True) for d in draws])

    def _build_model(self):
        model = Sequential()
        model.add(Input(shape=(self.window_size, self.input_size)))
        model.add(LSTM(self.units, return_sequences=False))
        model.add(Dense(self.range_max + 1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model = model

    def train(self, data: pd.DataFrame, epochs: int = 50, batch_size: int = 32, **kwargs):
        epochs = int(epochs)
        batch_size = int(batch_size)
        if 'window_size' in kwargs: self.window_size = int(kwargs['window_size'])
        if 'units' in kwargs: self.units = int(kwargs['units'])
        if self.units != 128 or self.model is None: self._build_model()
        print(f"Training LSTM: epochs={epochs}, batch={batch_size}, window={self.window_size}, units={self.units}")
        X, y = self._prepare_sequences(data)
        ball_cols = [c for c in data.columns if 'bola' in c.lower() or 'dezenas' in c.lower()]
        draws = data[ball_cols].values.tolist()
        if len(draws) >= self.window_size: self.last_window = draws[-self.window_size:]
        else: self.last_window = None
        if len(X) == 0: return
        # Verbose=0 para reduzir barulho no notebook
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=0)

    def predict(self, **kwargs) -> list:
        current_window = None
        if hasattr(self, 'last_window') and self.last_window is not None: current_window = self.last_window
        if current_window:
            X_input = np.array([self._draws_to_multihot(current_window)])
            if self.model is None: return []
            prediction = self.model.predict(X_input)
            probs = prediction[0]
            final_count = int(kwargs.get('count', self.draw_count) or self.draw_count)
            top_indices = probs.argsort()[-final_count:][::-1]
            return sorted([int(x) for x in top_indices])
        return []

## 5. Implementação das Loterias (Mega Sena, Lotofácil, Quina)

In [ ]:
# --- src/loterias/megasena.py ---
class MegaSena(Lottery):
    def __init__(self):
        super().__init__(
            name="Mega Sena",
            data_url="https://raw.githubusercontent.com/aretw0/loterias-caixa-db/refs/heads/main/data/megasena.csv",
            slug="megasena"
        )

    def load_data(self) -> pd.DataFrame:
        self.data = DataManager.load_csv(self.data_url)
        return self.data

    def preprocess_data(self) -> pd.DataFrame:
        if self.data is None:
            self.load_data()
        df = self.data.copy()
        df = df.rename(columns={'Data do Sorteio': 'data'})
        bola_cols = [f'Bola{i}' for i in range(1, 7)]
        df['dezenas'] = df[bola_cols].values.tolist()
        self.data = df
        return df

# --- src/loterias/lotofacil.py ---
class Lotofacil(Lottery):
    def __init__(self):
        super().__init__(
            name="Lotofácil",
            data_url="https://raw.githubusercontent.com/aretw0/loterias-caixa-db/refs/heads/main/data/lotofacil.csv",
            slug="lotofacil"
        )

    def load_data(self) -> pd.DataFrame:
        self.data = DataManager.load_csv(self.data_url)
        return self.data

    def preprocess_data(self) -> pd.DataFrame:
        if self.data is None:
            self.load_data()
        df = self.data.copy()
        df = df.rename(columns={'Data Sorteio': 'data'})
        bola_cols = [f'Bola{i}' for i in range(1, 16)]
        df['dezenas'] = df[bola_cols].values.tolist()
        self.data = df
        return df

# --- src/loterias/quina.py ---
class Quina(Lottery):
    def __init__(self):
        super().__init__(
            name="Quina",
            data_url="https://raw.githubusercontent.com/aretw0/loterias-caixa-db/refs/heads/main/data/quina.csv",
            slug="quina"
        )

    def load_data(self) -> pd.DataFrame:
        self.data = DataManager.load_csv(self.data_url)
        return self.data

    def preprocess_data(self) -> pd.DataFrame:
        if self.data is None:
            self.load_data()
        df = self.data.copy()
        df = df.rename(columns={'Data Sorteio': 'data'})
        bola_cols = [f'Bola{i}' for i in range(1, 6)]
        df['dezenas'] = df[bola_cols].values.tolist()
        self.data = df
        return df

## 6. Motor de Backtesting (Ensemble)

In [ ]:
# --- src/loterias/ensemble_backtester.py ---
class EnsembleBacktester:
    def __init__(self, lottery: Lottery, range_min: int, range_max: int, draw_count: int, model_args: Dict[str, Any] = None):
        self.lottery = lottery
        self.range_min = range_min
        self.range_max = range_max
        self.draw_count = draw_count
        self.model_args = model_args or {}
        
    def run(self, draws_to_test: int = 10, verbose: bool = True) -> Dict[str, Any]:
        # Ensure data is loaded
        df = self.lottery.preprocess_data()
        total_draws = len(df)
        if draws_to_test > total_draws: draws_to_test = total_draws
        start_index = total_draws - draws_to_test
        results = []
        
        rf_estimators = int(self.model_args.get('rf_n_estimators', self.model_args.get('n_estimators', 100)))
        xgb_estimators = int(self.model_args.get('xgb_n_estimators', self.model_args.get('n_estimators', 100)))
        lstm_epochs = int(self.model_args.get('epochs', 10))
        lstm_units = int(self.model_args.get('units', 128))
        
        print(f"[Backtest] Testando últimos {draws_to_test} concursos.")
        print(f"Config: RF={rf_estimators}, XGB={xgb_estimators}, LSTM={lstm_epochs} eps.")

        for i in range(start_index, total_draws):
            train_data = df.iloc[:i].copy()
            target_draw = df.iloc[i]
            target_numbers = set(target_draw['dezenas'])
            preds = {}
            
            # 1. Monte Carlo
            try:
                mc = MonteCarloModel(self.range_min, self.range_max, self.draw_count)
                mc.train(train_data)
                preds['mc'] = set(mc.predict())
            except Exception as e: print(f"Error MC: {e}"); preds['mc'] = set()

            # 2. Random Forest
            try:
                rf = RandomForestModel(self.range_min, self.range_max, self.draw_count)
                # Fix do n_jobs passado nos kwargs
                rf.train(train_data, n_estimators=rf_estimators, **self.model_args)
                preds['rf'] = set(rf.predict())
            except Exception as e: print(f"Error RF: {e}"); preds['rf'] = set()

            # 3. XGBoost
            try:
                xgb_model = XGBoostModel(self.range_min, self.range_max, self.draw_count)
                xgb_args = self.model_args.copy()
                xgb_args['n_estimators'] = xgb_estimators
                if 'rf_n_estimators' in xgb_args: del xgb_args['rf_n_estimators']
                xgb_model.train(train_data, **xgb_args)
                preds['xgb'] = set(xgb_model.predict())
            except Exception as e: print(f"Error XGB: {e}"); preds['xgb'] = set()

            # 4. LSTM
            try:
                lstm = LSTMModel(self.range_min, self.range_max, self.draw_count)
                # Fix de argumentos conflitantes
                lstm_args = self.model_args.copy()
                if 'epochs' in lstm_args: del lstm_args['epochs']
                if 'units' in lstm_args: del lstm_args['units']
                lstm.train(train_data, epochs=lstm_epochs, batch_size=32, units=lstm_units, **lstm_args)
                preds['lstm'] = set(lstm.predict())
            except Exception as e: print(f"Error LSTM: {e}"); preds['lstm'] = set()

            # Consensus
            all_votes = []
            for p in preds.values(): all_votes.extend(list(p))
            vote_counts = Counter(all_votes)
            con_4 = {n for n, c in vote_counts.items() if c >= 4}
            con_3 = {n for n, c in vote_counts.items() if c >= 3}
            con_2 = {n for n, c in vote_counts.items() if c >= 2}
            
            row_result = {
                'draw_index': i,
                'target': list(target_numbers),
                'hits': {
                    'mc': len(preds['mc'].intersection(target_numbers)),
                    'rf': len(preds['rf'].intersection(target_numbers)),
                    'xgb': len(preds['xgb'].intersection(target_numbers)),
                    'lstm': len(preds['lstm'].intersection(target_numbers)),
                    'con_4': len(con_4.intersection(target_numbers)),
                    'con_3': len(con_3.intersection(target_numbers)),
                }
            }
            results.append(row_result)
            
            print(f"Concurso {i} | Alvo: {list(target_numbers)}")
            print(f"  Acertos -> MC:{row_result['hits']['mc']} RF:{row_result['hits']['rf']} XGB:{row_result['hits']['xgb']} LSTM:{row_result['hits']['lstm']} | Consenso(3+): {row_result['hits']['con_3']}")
            
            del lstm, rf, xgb_model, mc
            tf.keras.backend.clear_session()
            gc.collect()

        return results

## 7. Execução (Main)

### Execução Mega Sena

In [ ]:
# Configuração da Execução Pesada
ARGS = {
    'game': 'megasena',
    'draws_to_backtest': 10,  # Quantos sorteios testar
    'model_args': {
        'rf_n_estimators': 2000,   # Alto custo
        'xgb_n_estimators': 1000,  # Alto custo
        'xgb_learning_rate': 0.01,
        'epochs': 500,             # Alto custo (LSTM)
        'units': 256,              # Neurônios LSTM
        'window_size': 15,         # Janela de tempo
        'n_jobs': -1               # Usar todas as CPUs do Colab (seguro lá)
    }
}

print("Iniciando Backtest com parâmetros:", ARGS)

# Inicializar Jogo
lottery = MegaSena()
game_config = {'min': 1, 'max': 60, 'draw': 6}

# Executar
backtester = EnsembleBacktester(
    lottery, 
    game_config['min'], 
    game_config['max'], 
    game_config['draw'], 
    model_args=ARGS['model_args']
)

final_results = backtester.run(draws_to_test=ARGS['draws_to_backtest'])

### Execução Lotofácil

In [ ]:
# Configuração para Lotofácil
ARGS_LOTOFACIL = {
    'game': 'lotofacil',
    'draws_to_backtest': 10,
    'model_args': {
        'rf_n_estimators': 2000,
        'xgb_n_estimators': 1000,
        'xgb_learning_rate': 0.01,
        'epochs': 300,             # Menos épocas pois tem mais dados
        'units': 256,
        'window_size': 20,
        'n_jobs': -1
    }
}

print("Iniciando Backtest Lotofácil :", ARGS_LOTOFACIL)

lottery = Lotofacil()
game_config = {'min': 1, 'max': 25, 'draw': 15}

backtester = EnsembleBacktester(
    lottery, 
    game_config['min'], 
    game_config['max'], 
    game_config['draw'], 
    model_args=ARGS_LOTOFACIL['model_args']
)

results_lf = backtester.run(draws_to_test=ARGS_LOTOFACIL['draws_to_backtest'])

### Execução Quina

In [ ]:
# Configuração para Quina
ARGS_QUINA = {
    'game': 'quina',
    'draws_to_backtest': 10,
    'model_args': {
        'rf_n_estimators': 2000,
        'xgb_n_estimators': 1000,
        'xgb_learning_rate': 0.01,
        'epochs': 500,
        'units': 256,
        'window_size': 15,
        'n_jobs': -1
    }
}

print("Iniciando Backtest Quina :", ARGS_QUINA)

lottery = Quina()
game_config = {'min': 1, 'max': 80, 'draw': 5}

backtester = EnsembleBacktester(
    lottery, 
    game_config['min'], 
    game_config['max'], 
    game_config['draw'], 
    model_args=ARGS_QUINA['model_args']
)

results_quina = backtester.run(draws_to_test=ARGS_QUINA['draws_to_backtest'])